# Collect data

Indexes we want:

*   NDVI
*   GCI
*   NDWI



## Clean data folder

In [1]:
# DANGER ZONE 2

# !rm -rf spaceapps-raw

## Install and import statements

In [3]:
import requests
import json
from pprint import pprint
import os
from datetime import datetime
import re
from tqdm import tqdm

## Generate authorization token

In [4]:
# Get authorization token

# EarthData AppEEARS credentials
username = "token_american"
password = "AerVJ&5CL8&8PP_"

# Request token
response = requests.post('https://appeears.earthdatacloud.nasa.gov/api/login', auth=(username, password))
token_response = response.json()
pprint(token_response)

# Store token
token = token_response["token"]

{'expiration': '2024-10-07T20:37:04Z',
 'token': 'N5gXbdnzl0aSzyhoZgoYVHwogxkH2y-zEbagG49fTRYf_GqkdZV0-M7XGoOhuu-7mUG-NJ-fKLhaRbUKsNULAQ',
 'token_type': 'Bearer'}


## Create Region Of Interest (ROI)

In [5]:
# Define the area of interest using a polygon (GeoJSON format)

polygon = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                       
                        [-121.3765, 36.8772],
                        [-121.3768, 36.8723],
                        [-121.3728, 36.8721],
                        [-121.3726, 36.8760],
                        [-121.3765, 36.8772]
                        
                        
                    ]
                ]
            }
        }
    ]
}

## Useful information about how to make requests

In [6]:
# # See all the different products that are possible to request
# response = requests.get('https://appeears.earthdatacloud.nasa.gov/api/product')
# product_response = response.json()

# # Create a dictionary indexed by the product name and version
# products = {p['ProductAndVersion']: p for p in product_response}

# # Filter the dictionary for keys that start with 'L'
# filtered_products = {k: v for k, v in products.items() if 'landsat' in v.get("Description", "").lower()}

# pprint(filtered_products)

In [7]:
# Check layers associated with the product

# product_id = 'L09.002'
# response = requests.get('https://appeears.earthdatacloud.nasa.gov/api/product/{0}'.format(product_id))
# layer_response = response.json()
# pprint(layer_response)

### Notes about Layers

* The QA_PIXEL layer would allow us to filter out bad pixels. Maybe we wouldn't need to filter out the images over the cloud cover threshold.

* Likewise, the QA_RADSAT layer flags pixels that are radiometrically saturated, which means they've reached the sensor's detectable limit.

* LandSAT 8 and 9 have temporal granularity of 16 days.

## Download data

### Create request

In [9]:
# Define the AppEEARS request

# Give start and end dates in MM-DD-YYYY
start_date = "10-31-2021"  # First available date for Landsat 9 imagery
end_date = "10-05-2024"

request_payload = {
    "task_type": "area",
    "task_name": "California_Field",
    "params": {
        # The last layers dictionary must not have a comma at the end
         "layers": [
            {"product": "L09.002", "layer": "SR_B2"},  # Blue band for raster plot of visible light
            {"product": "L09.002", "layer": "SR_B3"},  # Green band for NDWI and GCI
            {"product": "L09.002", "layer": "SR_B4"},  # Red band for NDVI, GCI
            {"product": "L09.002", "layer": "SR_B5"}  # NIR band for NDVI, GCI, NDWI
            # {"product": "L09.002", "layer": "SR_B6"},  # SWIR1 band for NDWI
            # {"product": "L09.002", "layer": "SR_B7"}  # SWIR2 band
         ],
        "dates": [{"startDate": start_date, "endDate": end_date}],
        "output": {"format": {"type": "geotiff"}, "projection": "geographic"},
        "geo": polygon,
        "cloud_cover": {"max": 30}  # Filter out images with more than 30% cloud cover
    }
}

### AppEEARS API Tasking

In [10]:
# Submit the task request

response = requests.post(
    'https://appeears.earthdatacloud.nasa.gov/api/task',
    json=request_payload,
    headers={'Authorization': 'Bearer {0}'.format(token)})
task_response = response.json()
print(task_response)

task_id = task_response["task_id"]

{'task_id': 'c707f90a-dc7b-44f8-a0ae-024c8f9c0797', 'status': 'pending'}


In [11]:
def delete_task(token, task_id):
  response = requests.delete(
    'https://appeears.earthdatacloud.nasa.gov/api/task/{0}'.format(task_id),
    headers={'Authorization': 'Bearer {0}'.format(token)})
  print(response.status_code)

In [21]:
# Checking task status

# # Checking status for all tasks assigned to our user
# response = requests.get(
#     'https://appeears.earthdatacloud.nasa.gov/api/status/',
#     headers={'Authorization': 'Bearer {0}'.format(token)})
# status_response = response.json()
# pprint(status_response)

# task_id = 'b9fcd494-cd4d-47e1-9191-bca101f0be6c'

# # Checking status for specific task
response = requests.get(
    'https://appeears.earthdatacloud.nasa.gov/api/status/{0}'.format(task_id),
    headers={'Authorization': 'Bearer {0}'.format(token)})
status_response = response.json()
pprint(status_response)

{'has_swath': False,
 'progress': {'details': [{'desc': 'Initializing',
                           'pct_complete': 100,
                           'step': 1},
                          {'desc': 'Downloading',
                           'pct_complete': 100,
                           'step': 2},
                          {'desc': 'Subsetting',
                           'pct_complete': 100,
                           'step': 3},
                          {'desc': 'Generating Files',
                           'pct_complete': 33,
                           'step': 4},
                          {'desc': 'Extracting Stats',
                           'pct_complete': 0,
                           'step': 5},
                          {'desc': 'Finalizing', 'pct_complete': 0, 'step': 6}],
              'summary': 58,
              'user_consider_limit': False},
 'status_id': '5e1efd57-befb-4ba5-818c-ec3a2d7aa266',
 'status_type': 'task',
 'task_id': 'c707f90a-dc7b-44f8-a0ae-024c8f9c0797',
 '

In [ ]:
# DANGER ZONE

# delete_task(token, '76e76e65-5753-4e00-a8a1-04555a47d182')

### Fetch & Format Files

In [ ]:
# Retrieve files

response = requests.get(
    'https://appeears.earthdatacloud.nasa.gov/api/bundle/{0}'.format(task_id),
    headers={'Authorization': 'Bearer {0}'.format(token)}
)
files_response = response.json()

# files_response has other data types included
pprint(files_response["files"][0:2])

[{'file_id': '966af9eb-cb1c-4e1b-8fc0-2850d53447e7',
  'file_name': 'L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021320_aid0001.tif',
  'file_size': 2012,
  'file_type': 'tif',
  's3_url': 's3://appeears-output/b9fcd494-cd4d-47e1-9191-bca101f0be6c/L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021320_aid0001.tif',
  'sha256': 'e74818ca627da3b09bc3666e3dbf9e7808ba8caceef69a4b86ba224fe4e79a72'},
 {'file_id': '209e953f-4d31-42c6-9773-7371c3db69a1',
  'file_name': 'L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021325_aid0001.tif',
  'file_size': 1913,
  'file_type': 'tif',
  's3_url': 's3://appeears-output/b9fcd494-cd4d-47e1-9191-bca101f0be6c/L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021325_aid0001.tif',
  'sha256': '6b249d1957dca19c33ed97a8b06654b51fd0acd5fecba5f23e4a25f883a9a23b'}]


In [ ]:
# # Select A FEW FILES to work with

# file_ids = [
#     'b85bc5ac-d8fb-48eb-a600-a143d55c1399',  # B2
#     'aeb0ae27-62ad-4fcb-9f45-4e2321cd9eb8',  # B3
#     'bf2ed48c-6151-4cac-9962-2c40f25906f0'  # B4
#     ]

# # # we are intentionally using 3 files only for now
# # for f in range(0, 3):
# #   file_ids.append(files_response["files"][f]["file_id"])

# pprint(file_ids)

# for file_id in file_ids:
#   filename = file_id + '.tif'
#   response = requests.get(
#       'https://appeears.earthdatacloud.nasa.gov/api/bundle/{0}/{1}'.format(task_id,file_id),
#       headers={'Authorization': 'Bearer {0}'.format(token)},
#       allow_redirects=True,
#       stream=True
#   )

#   # Create the filepath if it doesn't exist
#   filepath = os.path.join(dest_dir, filename)
#   os.makedirs(os.path.dirname(filepath), exist_ok=True)

#   # write the file to the destination directory
#   with open(filepath, 'wb') as f:
#       for data in response.iter_content(chunk_size=8192):
#           f.write(data)

In [ ]:
# Check if filename contains required patterns

def is_valid_filename(file_name):
    # Check for 'doy' pattern
    doy_match = re.search(r'doy\d+_', file_name)
    # Check for '_B#_' pattern
    band_match = re.search(r'_B\d+_', file_name)
    return bool(doy_match and band_match)

In [ ]:
# Format filename
def format_fname(file_name):
  if not is_valid_filename(file_name):
    raise ValueError(
        f"Could not format filename because it is invalid: {file_name}"
        )

  doy_match = re.search(r'doy\d+_', file_name)
  doy_string = doy_match.group(0).strip('_')

  band_match = re.search(r'_B\d+_', file_name)
  band_string = band_match.group(0).strip('_')

  return f"{doy_string}_{band_string}.tif"

In [ ]:
def download_file(file, dest_dir, token, task_id):
  filename = format_fname(file["file_name"])
  response = requests.get(
      'https://appeears.earthdatacloud.nasa.gov/api/bundle/{0}/{1}'.format(task_id, file["file_id"]),
      headers={'Authorization': 'Bearer {0}'.format(token)},
      allow_redirects=True,
      stream=True
  )

  filepath = os.path.join(dest_dir, filename)
  os.makedirs(os.path.dirname(filepath), exist_ok=True)

  # write the file to the destination directory
  with open(filepath, 'wb') as f:
      for data in response.iter_content(chunk_size=8192):
          f.write(data)

In [ ]:
# Download files

# Create directory where files will be stored
base_dir = "spaceapps-raw"

# Get the current date and time
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Create a subdirectory with the date and time
dest_dir = os.path.join(base_dir, current_time)

# Create the directory if it doesn't exist
os.makedirs(dest_dir, exist_ok=True)

skipped_files = []
# Download each file
for file in tqdm(
    files_response["files"],
    desc="Downloading image files",
    unit="file"
    ):
  if is_valid_filename(file["file_name"]):
    download_file(file, dest_dir, token, task_id)
  else:
    skipped_files.append(f"{file['file_name']}")

if skipped_files:
  print("\n\nSkipped these files on account of their names:")
  print("\n".join(skipped_files))



Skipped these files on account of their names:
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021320_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021325_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021337_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021344_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021353_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2021360_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2022004_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2022011_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2022020_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2022027_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2022036_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU_doy2022043_aid0001.tif
L09.002_2021289_to_2024279/L09.002_SR_QA_AEROSOL_CU

In [ ]:
# Zip the files to a folder for offline download
!zip -r /content/images.zip spaceapps-raw/2024-10-06_13-49-16
# dest_dir

  adding: spaceapps-raw/2024-10-06_13-49-16/ (stored 0%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2022164_B5.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2024033_B3.tif (deflated 9%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2024225_B4.tif (deflated 9%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2024209_B5.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2023263_B4.tif (deflated 9%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2022260_B4.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2023318_B4.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2023295_B5.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2024097_B4.tif (deflated 11%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2023199_B4.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2022340_B4.tif (deflated 8%)
  adding: spaceapps-raw/2024-10-06_13-49-16/doy2021360_B5.tif (deflated 10%)
  adding: spaceapps-raw/2024-

In [ ]:
# # Download the README and Statistics files

# meta_file_idxs = [-1, -5]

# meta_file_ids = [
#     files_response["files"][meta_file_idxs[0]]["file_id"],
#     files_response["files"][meta_file_idxs[1]]["file_id"]
# ]

# meta_fns = [
#     files_response["files"][meta_file_idxs[0]]["file_name"],
#     files_response["files"][meta_file_idxs[1]]["file_name"]
#     ]

# pprint(meta_fns)

# for i, fid in enumerate(meta_file_ids):
#   print(i, fid)
#   response = requests.get(
#       'https://appeears.earthdatacloud.nasa.gov/api/bundle/{0}/{1}'.format(task_id, fid),
#       headers={'Authorization': 'Bearer {0}'.format(token)},
#       allow_redirects = True,
#       stream=True
#   )

#   # Create the filepath if it doesn't exist
#   filename = meta_fns[i]
#   filepath = os.path.join(dest_dir, filename)
#   os.makedirs(os.path.dirname(filepath), exist_ok=True)

#   # write the file to the destination directory
#   with open(filepath, 'wb') as f:
#       for data in response.iter_content(chunk_size=8192):
#           f.write(data)

# Analyze data

In [ ]:
!pip install rasterio

In [ ]:
# import earthpy as et
# import earthpy.spatial as es
# import earthpy.plot as ep
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
# from matplotlib.colors import ListedColormap
# import plotly.graph_objects as go

In [ ]:
# Plot visual spectrum as a data quality check

# Consider the reflectance scale factor from layer metadata
scale_factor = 2.75e-05

# Open the GeoTIFF files for each band
with rio.open('/content/spaceapps-raw/b85bc5ac-d8fb-48eb-a600-a143d55c1399.tif') as blue_band, \
     rio.open('/content/spaceapps-raw/aeb0ae27-62ad-4fcb-9f45-4e2321cd9eb8.tif') as green_band, \
     rio.open('/content/spaceapps-raw/bf2ed48c-6151-4cac-9962-2c40f25906f0.tif') as red_band:

    # Read each band
    blue = blue_band.read(1).astype('float32') * scale_factor
    green = green_band.read(1).astype('float32') * scale_factor
    red = red_band.read(1).astype('float32') * scale_factor

    # Stack bands into an RGB image (Red, Green, Blue)
    rgb_image = np.dstack((red, green, blue))

    # Plot the RGB image using matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_image)
    plt.title("RGB Composite of SR_B2 (Blue), SR_B3 (Green), SR_B4 (Red)")
    plt.axis('off')  # Hide axis for better visual representation
    plt.show()

RasterioIOError: './spaceapps-raw/b85bc5ac-d8fb-48eb-a600-a143d55c1399.tif' not recognized as being in a supported file format.

In [ ]:
# prompt: Calculate NDVI

with rio.open('/content/spaceapps-raw/bf2ed48c-6151-4cac-9962-2c40f25906f0.tif') as red_band, \
     rio.open('/content/spaceapps-raw/aeb0ae27-62ad-4fcb-9f45-4e2321cd9eb8.tif') as nir_band:

    # Read each band
    red = red_band.read(1).astype('float32') * scale_factor
    nir = nir_band.read(1).astype('float32') * scale_factor

    # Calculate NDVI
    ndvi = (nir - red) / (nir + red)

    # Plot NDVI
    plt.figure(figsize=(10, 10))
    plt.imshow(ndvi, cmap='RdYlGn')
    plt.colorbar(label='NDVI')
    plt.title('Normalized Difference Vegetation Index (NDVI)')
    plt.axis('off')
    plt.show()


In [ ]:
# prompt: calculate GCI

with rio.open('/content/spaceapps-raw/aeb0ae27-62ad-4fcb-9f45-4e2321cd9eb8.tif') as green_band, \
     rio.open('/content/spaceapps-raw/bf2ed48c-6151-4cac-9962-2c40f25906f0.tif') as red_band:

    # Read each band
    green = green_band.read(1).astype('float32') * scale_factor
    red = red_band.read(1).astype('float32') * scale_factor

    # Calculate GCI
    gci = (nir / green) - 1

    # Plot GCI
    plt.figure(figsize=(10, 10))
    plt.imshow(gci, cmap='RdYlGn')
    plt.colorbar(label='GCI')
    plt.title('Green Chlorophyll Index (GCI)')
    plt.axis('off')
    plt.show()


# What should we do now?

1. Label 100 images
  - date_color.tif
1. Rotate image
1. Add computations for 3 indexes
1. Load more data..? Let's do 2 years for now.
1. Add data processing.

## Manny's Plan

```
for each image:
1. Compute the mean NDVI, GCI and Water Index
2.

```

